<a href="https://colab.research.google.com/github/lucianomcsilva/RIWRS/blob/main/%5BRIWRS%5D_3_ROBO_AJ_via_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurando o Scrapy


In [36]:
# Instalando bibliotecas
try:
    import scrapy
    from crochet import setup, wait_for
except:
    !pip install scrapy
    !pip install crochet
    import scrapy
    from scrapy.crawler import CrawlerRunner
    from crochet import setup, wait_for

In [37]:
# text cleaning
import re
import logging
# Reactor restart
setup()

# Montando a Aranha

In [38]:
class SpiderAj2Spider(scrapy.Spider):
    name = "spider_aj"
    delimiter = ';'
    
    # abre o arquivo local, com todos os links
    # start_urls = [f"{pathlib.Path(os.path.abspath('arnaldo_jabor.html')).as_uri()}"]
    start_urls = ["https://raw.githubusercontent.com/lucianomcsilva/robo_aj/master/arnaldo_jabor.html"]
    
    custom_settings = {
        'LOG_LEVEL': 'DEBUG',
        # 'FEED_URI': 'robo_aj.csv',                       # Used for pipeline     
        # 'FEED_FORMAT':'csv',                            # Used for pipeline
        'FEEDS': {
            'robo_aj.csv': {
                'format': 'csv',
                'overwrite': True
            }
        },                
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    
    # # Apaga o arquivo caso exista, para evitar sobreposição
    # if os.path.exists("artigos_aj.csv"):
    #     os.remove("artigos_aj.csv")

    # função que interpreta o documento que lista os artigos
    def parse(self, response):           
        for link in response.css(".item-ultimas").css("h2").css("a::attr(href)").getall():
            text_page = f"https://www.otempo.com.br/{link}"
            print(text_page)
            yield scrapy.Request(text_page, callback=self.parse_text)

    # função que interpreta os documentos com os textos 
    def parse_text(self, response):
        content = ""   
        #Atualizando para novo leiaute do site o tempo
        for line in response.css('#text-content p::text').getall() :
            content = content + "".join(line.strip()) + "\n"
            
        post = {
            'author': 'Arnaldo Jabor',
            'title': response.css('h1::text').get().strip().encode('utf-8'),   #Use strip para evitar novas lihas no titulo
            'content': content.encode('utf-8')
        }
        #yield {'author': 'Arnaldo Jabor',}
        yield post
        
        # with open('artigos_aj.csv', 'a', newline='', encoding="utf-8")  as output_file:
        #      dict_writer = csv.DictWriter(output_file, post.keys())            
        #      dict_writer.writerows([post])        
        # yield post

# Executando o Crawler

In [39]:
@wait_for(10)
def run_spider():
  setup()
  crawler = CrawlerRunner()
  crawler.crawl(SpiderAj2Spider)


In [40]:
run_spider()


It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.


# Carrega e mostra no Pandas

In [43]:
import pandas as pd

df = pd.read_csv("robo_aj.csv")
df

,author,title,content
0,Arnaldo Jabor,O militante imaginário,O que é o “militante imaginário”? O filósofo J...
1,Arnaldo Jabor,A volta da caretice,"Fala-se muito em esquerda e direita, mas nos e..."
2,Arnaldo Jabor,Entrevista comigo mesmo,"Oi, Arnaldo, tudo bem?\n– Tudo bem, Arnaldo......"
3,Arnaldo Jabor,"O exército do 'Fora, Temer'",Eu também quero ser feliz. Fico com inveja dos...
4,Arnaldo Jabor,Rio e terror,Venho por esta colocar minha colher na sopa de...
...,...,...,...
137,Arnaldo Jabor,Nossa vida será um videogame?,"era jovem, feliz, perto do selvagem coração da..."
138,Arnaldo Jabor,O mistério do gambá gigante,"Há seis anos, escrevi um artigo que repito, em..."
139,Arnaldo Jabor,"A miséria já teve uma ""função social""","Falar da miséria nos alivia porque para nós, o..."
140,Arnaldo Jabor,Os grandes arrepios,"Hoje começa o circo da propaganda eleitoral, o..."
